In [1]:
import sagemaker
from keras_to_tensorflow_pb import keras_to_pb

In [2]:
# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo2/' + model_version
model_path = '/home/ec2-user/SageMaker/keras_model/'+'2019_04_23_14_35_inception_v3_retrain.json'

In [3]:
keras_to_pb(model_path, export_dir)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
('load model ', '/home/ec2-user/SageMaker/keras_model/2019_04_23_14_35_inception_v3_retrain')
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo2/1/saved_model.pb


In [5]:
import tarfile
with tarfile.open('keras_model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [6]:
sagemaker_session = sagemaker.Session()

In [7]:
inputs = sagemaker_session.upload_data(path='keras_model.tar.gz', key_prefix='model')

In [8]:
!touch train.py

In [9]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/keras_model.tar.gz',
                                  role = 'arn:aws:iam::575544372592:role/service-role/AmazonSageMaker-ExecutionRole-20190503T142702',
                                  framework_version = '1.12',
                                  py_version='py2',
                                  entry_point = 'train.py')

No handlers could be found for logger "sagemaker"


In [10]:
predictor = sagemaker_model.deploy(initial_instance_count=1, 
                                   instance_type='ml.t2.medium')

--------------------------------------------------------------------------------------------------!

In [12]:
endpoint_name = predictor.endpoint

In [13]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

### Invoke the SageMaker endpoint using a boto3 client

In [16]:
import json
import boto3
import numpy as np
import io
from model_predict_proba import tensor_from_images_list
 
client = boto3.client('runtime.sagemaker')
# The sample model expects an input of shape [1,50]
img_path = 'GameplayCategory0853.jpg'
data = tensor_from_images_list([img_path], (224, 224, 3))


In [17]:
predictor.predict(data)

{'model_spec': {'name': u'generic_model',
  'signature_name': u'serving_default',
  'version': {'value': 1L}},
 'outputs': {u'score': {'dtype': 1,
   'float_val': [1.5514427786911256e-06,
    2.634675463908387e-11,
    0.9999984502792358],
   'tensor_shape': {'dim': [{'size': 1L}, {'size': 3L}]}}}}

In [20]:
response = client.invoke_endpoint(EndpointName=endpoint_name, Body=json.dumps(data.tolist()))
response_body = response['Body']
print(response_body.read())

{
  "outputs": {
    "score": {
      "dtype": "DT_FLOAT", 
      "floatVal": [
        1.5514427786911256e-06, 
        2.634675463908387e-11, 
        0.9999984502792358
      ], 
      "tensorShape": {
        "dim": [
          {
            "size": "1"
          }, 
          {
            "size": "3"
          }
        ]
      }
    }
  }, 
  "modelSpec": {
    "version": "1", 
    "name": "generic_model", 
    "signatureName": "serving_default"
  }
}
